<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CTR-prediction" data-toc-modified-id="CTR-prediction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CTR-prediction</a></span><ul class="toc-item"><li><span><a href="#Problem-Formulation" data-toc-modified-id="Problem-Formulation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Problem Formulation</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Dataset-construction:" data-toc-modified-id="Dataset-construction:-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Dataset construction:</a></span></li><li><span><a href="#Format:" data-toc-modified-id="Format:-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Format:</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></li><li><span><a href="#Dataset-preprocessing" data-toc-modified-id="Dataset-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset preprocessing</a></span><ul class="toc-item"><li><span><a href="#ML-Pipelines-(Transformers,-Estimators)" data-toc-modified-id="ML-Pipelines-(Transformers,-Estimators)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components" target="_blank">ML Pipelines (Transformers, Estimators)</a></a></span><ul class="toc-item"><li><span><a href="#Prepare-stages-of-pipeline" data-toc-modified-id="Prepare-stages-of-pipeline-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Prepare stages of pipeline</a></span></li><li><span><a href="#Fit-and-save-pipeline" data-toc-modified-id="Fit-and-save-pipeline-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Fit and save pipeline</a></span></li><li><span><a href="#Load-fitted-pipeline" data-toc-modified-id="Load-fitted-pipeline-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Load fitted pipeline</a></span></li><li><span><a href="#Transform-dataset-using-pipeline" data-toc-modified-id="Transform-dataset-using-pipeline-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Transform dataset using pipeline</a></span></li><li><span><a href="#Make-dataset-split" data-toc-modified-id="Make-dataset-split-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Make dataset split</a></span></li></ul></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html" target="_blank">Classification</a></a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression" target="_blank">Logistic Regression</a></a></span><ul class="toc-item"><li><span><a href="#Define-and-Train-model" data-toc-modified-id="Define-and-Train-model-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Define and Train model</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html" target="_blank">Evaluation</a></a></span><ul class="toc-item"><li><span><a href="#Binary-classification-metrics" data-toc-modified-id="Binary-classification-metrics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification" target="_blank">Binary classification metrics</a></a></span></li><li><span><a href="#Make-submission" data-toc-modified-id="Make-submission-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Make submission</a></span></li></ul></li></ul></div>

# CTR-prediction

## Problem Formulation

$\newcommand{\vecw}{{\bf w}}$
$\newcommand{\vecx}{{\bf x}}$

* Dataset: $X^N = \{ z_i \}^N_{i=1}$, где $z_i = (\vecx_i, y_i) \sim P(z), y_i \in \{0,1\}$
* Prediction: $$ \hat{y}_i = f_{\vecw}(\vecx_i) =  \mathbb{P} \left\{ y = 1 \mid \vecx_i \right\} $$
* Loss function (Binary Cross-Entropy): $$ \min\limits_{\vecw} \quad \frac{\lambda}{2}\| \vecw \|^2_2 - \frac{1}{N} \sum\limits_{i=1}^{N} y_i \log \hat{y}_i + (1-y_i) \log(1-\hat{y}_i) $$

## Dataset
$ $
<details>
  <summary>Click here to see the details</summary>

For more details see `/data/criteo/readme.txt`

### Dataset construction:


>There are 13 features taking **integer** values and 26
**categorical** features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
Some features may have missing values.

> The rows are chronologically ordered by `id` column.

> The test set corresponds to events on the day following the training period. 
The first column (`label`) has been removed.


### Format:

> The columns are comma separeted with the following schema:
`<label>,<integer feature 1>, ... <integer feature 13>,<categorical feature 1>, ... <categorical feature 26>,<id>`

> When a value is missing, the field is "". There is no `label` field in the test set.

</details>
    
## Metrics

The evaluation metrics for this task are
* ROC AUC
* LogLoss
* [Normalized Entropy](https://quinonero.net/Publications/predicting-clicks-facebook.pdf)

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

Lets begin our introduction to Spark [MLlib](https://spark.apache.org/docs/latest/ml-guide.html)

---
# Dataset preprocessing

Before we can train any prediction model on our dataset we need to conver each row into real-valued features vector ($\vecx \in \mathbb{R}^n$).

Spark MLlib provides easy to use tools for preprocessing raw features and turning them into suitable format.

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

## to fix the sample

df = df.sample(False, 0.1)

df.select('id').toPandas().to_csv('./sample_ids.csv', index=None)

load sample

In [4]:
import pandas as pd

sample_ids = spark.createDataFrame(pd.read_csv('./sample_ids.csv')[['id']])

df = df.join(
        sample_ids,
        on='id',
        how='inner'
    )

Keeping only first two categorical features for simplicity

In [5]:
cnt_feats = df.columns[2:15]
hash_feats = df.columns[15:]
label = df.columns[1]

In [ ]:
df = df\
    .fillna(0, subset=cnt_feats)\
    .fillna('none', subset=hash_feats)

In [10]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: string (nullable = false)
 |-- _c15: string (nullable = false)
 |-- _c16: string (nullable = false)
 |-- _c17: string (nullable = false)
 |-- _c18: string (nullable = false)
 |-- _c19: string (nullable = false)
 |-- _c20: string (nullable = false)
 |-- _c21: string (nullable = false)
 |-- _c22: string (nullable = false)
 |-- _c23: string (nullable = false)
 |-- _c24: string (nullable = false)
 |-- _c25: string (nullable = false)
 |-- _c26: 

## [ML Pipelines (Transformers, Estimators)](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components)


MLlib standardizes APIs for machine learning algorithms to make it easier to combine multiple algorithms into a single pipeline, or workflow.

* `Transformer`: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


* `Estimator`: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


* `Pipeline`: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

---
Basically speaking `transformer` is an instance of class that implements `transform` method, and both `estimator` and `pipeline` implements `transform` and `fit` methods.

---

### Prepare stages of pipeline

We might benefit from using `StringIndexer, OneHotEncoderEstimator, VectorAssembler` (see [doc](https://spark.apache.org/docs/latest/ml-features) for details) 

The concept behind String Indexing is very intuitive. We simply replace each category with a number. 

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, StandardScaler

In [15]:
indexers = [StringIndexer(inputCol=col, outputCol=col+'_index') for col in hash_feats]

ohe = OneHotEncoderEstimator(
    inputCols=[col+'_index' for col in hash_feats[:3]], 
    outputCols=[col+'_ohe' for col in hash_feats[:3]]
)

ohe_assembler = VectorAssembler(inputCols=[col+'_ohe' for col in hash_feats[:3]], outputCol='ohe_features')

num_assembler = VectorAssembler(inputCols=cnt_feats, outputCol="num_features")

scaler = StandardScaler(inputCol="num_features", outputCol="scaled_features")

final_assembler = VectorAssembler(inputCols=['scaled_features', 'ohe_features'], outputCol='features')

### Fit and save pipeline

In [16]:
pipeline  = Pipeline(stages=indexers + [ohe, ohe_assembler, num_assembler, scaler, final_assembler])

In [17]:
pm = pipeline.fit(df)
pm.save("/pipeline_model_test")

### Load fitted pipeline

In [18]:
from pyspark.ml import PipelineModel

In [19]:
pm_l = PipelineModel.load("/pipeline_model_test")

### Transform dataset using pipeline

In [20]:
transformed_data = pm_l.transform(df)

transformed_data = transformed_data.select('features', 'id', label)

In [23]:
transformed_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- id: long (nullable = true)
 |-- _c0: integer (nullable = true)



In [27]:
transformed_data.write.parquet('trans_data_1')

In [28]:
transformed_data.show(3)

+--------------------+-----+---+
|            features|   id|_c0|
+--------------------+-----+---+
|(149520,[2,3,4,5,...|49586|  1|
|(149520,[1,2,4,5,...|49967|  0|
|(149520,[1,2,3,4,...|52611|  0|
+--------------------+-----+---+
only showing top 3 rows



### Make dataset split

Spark provides [randomSplit](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) method.

It is not the best choice in our task since we have chronological order in data.

We need to implement our own split function which will split the data in parts with respect to chronological order.

In [29]:
def split_by_col(df, split_col, parts_fractions, N):
    """
    df - DataFrame
    split_col - total order column
    parts_fractions - fractions of resulting parts
    """
    
    df = df.sort(split_col, ascending=True)
    
    rdd_df = df.select(split_col).rdd.zipWithIndex()
    df_final = rdd_df.toDF()
    df_final = df_final\
        .withColumn(split_col, df_final['_1'].getItem(split_col))\
        .drop('_1')
    
    df_final = df.join(
                        df_final.withColumnRenamed('_1', 'id'),
                        how='left',
                        on=split_col
                        )
    
    train, val = round(parts_fractions[0] * N), round(sum(parts_fractions[:2]) * N)

    
    parts = []
    
    df_final = df_final.sort(split_col, ascending=True)\
        .withColumnRenamed('_2', 'idx_row')\
        .withColumn('fractions', 
                    F.when(F.col('idx_row') < F.lit(train), 'train')\
                        .otherwise(F.when((F.col('idx_row') >= F.lit(train)) & (F.col('idx_row') < F.lit(val)), 'val')\
                                   .otherwise('test'))
                   )
    parts = [df_final\
             .filter(F.col('fractions') == F.lit(frac)) for frac in ['train', 'val', 'test']]
    
    return parts

In [30]:
N = transformed_data.count()

In [31]:
train_df, val_df, test_df = split_by_col(transformed_data, 'id', [0.8, 0.1, 0.1], N)

In [32]:
train_df.count() / N, val_df.count() / N, test_df.count() / N

(0.8, 0.1, 0.1)

In [36]:
train_df.count()

293360

In [39]:
val_df.show(3)

+------------+--------------------+---+-------+---------+
|          id|            features|_c0|idx_row|fractions|
+------------+--------------------+---+-------+---------+
|455266621063|(149520,[0,1,2,3,...|  1| 293360|      val|
|455266621067|(149520,[1,4,5,6,...|  0| 293361|      val|
|455266621132|(149520,[2,4,5,6,...|  0| 293362|      val|
+------------+--------------------+---+-------+---------+
only showing top 3 rows



In [38]:
train_df.cache()
val_df.cache()
test_df.cache()

DataFrame[id: bigint, features: vector, _c0: int, idx_row: bigint, fractions: string]

---
# [Classification](https://spark.apache.org/docs/latest/ml-classification-regression.html)

## [Logistic Regression](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression)

### Define and Train model

In [40]:
from pyspark.ml.classification import LogisticRegression 
lr_model = LogisticRegression(featuresCol='features', labelCol=label)

In [48]:
from pyspark.ml.classification import LogisticRegressionModel

модель училась ровно 2 часа

293360 примеров

и пришлось поставить ("spark.driver.memory", "4g")

In [41]:
lr_fitted = lr_model.fit(train_df)

In [44]:
lr_fitted.save(os.path.join(DATA_PATH, 'logreg.model'))

In [49]:
final_model = LogisticRegressionModel.load(os.path.join(DATA_PATH, 'logreg.model'))

In [50]:
val_df_pred = final_model.transform(val_df)

In [52]:
test_df_pred = final_model.transform(test_df)

In [51]:
val_df_pred.show(5)

+------------+--------------------+---+-------+---------+--------------------+--------------------+----------+
|          id|            features|_c0|idx_row|fractions|       rawPrediction|         probability|prediction|
+------------+--------------------+---+-------+---------+--------------------+--------------------+----------+
|455266621063|(149520,[0,1,2,3,...|  1| 293360|      val|[-0.1571897065234...|[0.46078328931603...|       1.0|
|455266621067|(149520,[1,4,5,6,...|  0| 293361|      val|[0.90381419637698...|[0.71173269004843...|       0.0|
|455266621132|(149520,[2,4,5,6,...|  0| 293362|      val|[2.06544360603564...|[0.88749883245489...|       0.0|
|455266621136|(149520,[0,1,3,4,...|  0| 293363|      val|[1.49746315413032...|[0.81719580976793...|       0.0|
|455266621151|(149520,[1,4,6,10...|  0| 293364|      val|[3.26033633883637...|[0.96304276333765...|       0.0|
+------------+--------------------+---+-------+---------+--------------------+--------------------+----------+
o

---
# [Evaluation](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html)

## [Binary classification metrics](https://spark.apache.org/docs/2.1.0/mllib-evaluation-metrics.html#binary-classification)

* ROC AUC
* LogLoss
* Normalized Entropy

In [92]:
import numpy as np

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

second_el = udf(lambda v: float(v[1]), FloatType())

In [139]:
from pyspark.sql.types import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics


def rocauc(model, df, label_name):
    
    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    
    evaluator = BinaryClassificationEvaluator(labelCol=label_name)
    
    return evaluator.evaluate(model.transform(df), {evaluator.metricName: "areaUnderROC"})
    


def logloss(model, df, label_name):
    #-(yt log(yp) + (1 - yt) log(1 - yp))
    
    df1 = model.transform(df)\
        .withColumn('logloss', -(F.col(label_name) * F.log(second_el('probability'))\
                       + (1 -  F.col(label_name)) * F.log(1 - second_el('probability'))) 
                   )\
        .select(F.mean('logloss').alias('logloss'))

    
    return df1.take(1)[0]['logloss']


def ne(df, logloss, label_name):
    
    mean_lit = df\
        .select(F.mean(label_name).alias('mean_lab')).take(1)[0]['mean_lab']
    
    mean_logloss = -(mean_lit * np.log(mean_lit) + (1 - mean_lit)*np.log(1 - mean_lit))
        
    return logloss / mean_logloss

In [126]:
logloss(final_model, test_df, label)

0.7789764836462141

In [128]:
rocauc(final_model, test_df, label)

0.6447082898932033

In [140]:
ne(test_df, 0.7789764836462141, label)

1.3437496925855579

## Make submission

Join the [competition](https://www.kaggle.com/c/mlbd-20-ctr-prediction-1) and make a submission

In [141]:
DATA_PATH = '/workspace/data/criteo'

TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

In [143]:
TEST = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

In [144]:
TEST = TEST\
    .fillna(0, subset=cnt_feats)\
    .fillna('none', subset=hash_feats)

### сколько категорий отсутсвтвуют в трейне и в обученных пайплайнах

In [160]:
for col in hash_feats[:3]:
    print(col)
    print(TEST.select(F.countDistinct(col)).take(1), df.select(F.countDistinct(col)).take(1))

_c14
[Row(count(DISTINCT _c14)=1240)] [Row(count(DISTINCT _c14)=909)]
_c15
[Row(count(DISTINCT _c15)=547)] [Row(count(DISTINCT _c15)=536)]
_c16
[Row(count(DISTINCT _c16)=324499)] [Row(count(DISTINCT _c16)=148065)]


In [171]:
def handle_invalid_keep(df, sample, col):
    """заменяет незнакомые значения на 'none'"""
    
    print('column', col)
    print('data count distinct', df.select(F.countDistinct(col).alias('cd')).take(1)[0]['cd'], 
          'sample count distinct', sample.select(F.countDistinct(col).alias('cd')).take(1)[0]['cd'])
    
    sample = sample.select(col).drop_duplicates().withColumn('keep', F.lit(1))
    df = df.join(
                sample,
                on=col,
                how='left'
            )\
        .fillna(0, subset=['keep'])\
        .withColumnRenamed(col, col+'_old')\
        .withColumn(col, 
                    F.when(F.col('keep') == F.lit(1), F.col(col+'_old'))\
                        .otherwise('none')
                   )\
        .drop(col+'_old', 'keep')
    
    return df

In [182]:
def handle_invalid_skip(df, col):
    """удаляет незнакомые значения"""
    
    print('size before', df.count())
    df1 = df.filter(F.col(col) != F.lit('none'))
    print('size after', df1.count())
    
    return df1

In [172]:
for col in   hash_feats[:3]:  
    TEST = handle_invalid_keep(TEST, df, col)

column _c14
data count distinct 1240 sample count distinct 909
column _c15
data count distinct 547 sample count distinct 536
column _c16
data count distinct 324499 sample count distinct 148065


In [173]:
for col in hash_feats[:3]:
    print(col)
    print(TEST.select(F.countDistinct(col)).take(1), df.select(F.countDistinct(col)).take(1))

_c14
[Row(count(DISTINCT _c14)=828)] [Row(count(DISTINCT _c14)=909)]
_c15
[Row(count(DISTINCT _c15)=529)] [Row(count(DISTINCT _c15)=536)]
_c16
[Row(count(DISTINCT _c16)=26291)] [Row(count(DISTINCT _c16)=148065)]


In [185]:
TEST_skipped = TEST

In [186]:
for col in   hash_feats[:3]:  
    TEST_skipped = handle_invalid_skip(TEST_skipped, col)

size before 917961
size after 917027
size before 917027
size after 916406
size before 916406
size after 539934


### только к 60% данных из теста можно применить модель

In [190]:
transformed_TEST = pm_l.transform(TEST_skipped.select(*hash_feats[:3], *cnt_feats, 'id'))

transformed_TEST = transformed_TEST.select('features', 'id')

In [191]:
pred_TEST = q1.transform(transformed_TEST)

pred_TEST.show(10)

+--------------------+------------+--------------------+--------------------+----------+
|            features|          id|       rawPrediction|         probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
|(149520,[0,1,2,3,...|704375096151|[11.1356513272780...|[0.99998541717235...|       0.0|
|(149520,[1,2,4,5,...|661425083742|[11.5303190800824...|[0.99999017252877...|       0.0|
|(149520,[1,2,3,4,...|575525906360|[11.5606574813648...|[0.99999046619843...|       0.0|
|(149520,[2,4,5,6,...|575525934899|[11.5700106962885...|[0.99999055495357...|       0.0|
|(149520,[1,2,4,5,...|618475422405|[12.2145288630303...|[0.99999504212740...|       0.0|
|(149520,[1,2,3,4,...|618475423709|[11.5011491787689...|[0.99998988164338...|       0.0|
|(149520,[1,2,3,4,...|618475575902|[11.3393488245236...|[0.99998810462299...|       0.0|
|(149520,[1,4,7,8,...|618475686165|[11.6380256759929...|[0.99999117599315...|       0.0|
|(149520,[1,2,3,4,...

### средний таргет на предсказаниях 0.11

In [192]:
pred_TEST.select(F.mean(F.col('prediction'))).take(1)

[Row(avg(prediction)=0.11114506587842218)]

### средний таргет на трейне 0.25

* заполнение оставшихся 40% данных средним таргетом из трейна

In [196]:
mean_lit = 0.2543632397054813

In [198]:
TEST_const = TEST.filter((F.col('_c14') == 'none') | (F.col('_c15') == 'none') | (F.col('_c16') == 'none'))\
    .withColumn('proba', F.lit(mean_lit))

In [199]:
result = pred_TEST\
    .select('id', second_el('probability').alias('proba'))\
    .union(TEST_const.select('id', 'proba'))

In [201]:
result.sort('id').toPandas().to_csv('./answers.csv', index=None)